# Retention Score Test - Redshift
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Mar 23, 2022

In [1]:
import boto3
import getpass
import pyodbc
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt


In [2]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

JRQN-CQLM
Successully logged into Start URL: https://stellaralgo.awsapps.com/start#/


### First we have to tell AWS which profile we'd like to login to, this will send us for a browser authentication trip:

### Connect to Redshift:

In [3]:
session = boto3.session.Session(profile_name='Stellaralgo-DataScienceAdmin')
client = session.client('redshift')

CLUSTER = 'qa-app'
DBNAME = 'stlrmilb'

creds2 = client.get_cluster_credentials(                
    ClusterIdentifier = CLUSTER,
    DbUser = 'admin',
    DbName = DBNAME,
    DbGroups = ['admin_group'],
    AutoCreate=True
)

# create a connection & cursor and call the stored procedure
conn = psycopg2.connect(
    host = 'qa-app.ctjussvyafp4.us-east-1.redshift.amazonaws.com',
    port = 5439,
    user = creds2['DbUser'],
    password = creds2['DbPassword'],
    database = DBNAME
)

print(f"GOT CONNECTION TO DATABASE: {CLUSTER} {DBNAME}")


GOT CONNECTION TO DATABASE: qa-app stlrmilb


### Create a dictionary of all milb teams:

In [13]:
# create a list of games dates 
lkupclientids= [9,11,12,15,19,20,24,26,27,28,30,34,40,43,44,47,48,49,51,59]

all_milb =[]
#df_all =pd.DataFrame()
for lkupclientid in lkupclientids:
    
    cur = conn.cursor()
    cur.execute(f"CALL stlrmilb.dw.getproductpropensitymodeldata({lkupclientid}, 2010, 2022, 'result')")

    # create a named cursor based on the cursor name passed in above
    named_cursor = conn.cursor('result')
    data = named_cursor.fetchall()

    # load the data and columns into a data frame
    cols = [row[0] for row in named_cursor.description]
    df = pd.DataFrame(data=data, columns=cols)
    df = df.fillna(0)
    all_milb.append(df)
   # df_all = df_all + df
    
    conn.commit()

In [14]:
df_all

,lkupclientid,dimcustomermasterid,distance,seasonyear,events_prior,attended_prior,events_last,attended_last,tenure,atp_last,product_current
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
646735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#df_all['product_current'].value_count()

TypeError: list indices must be integers or slices, not str

In [11]:
#df_all = pd.DataFrame(all_milb)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/internals/construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


In [19]:
all_products =[]
for clients in all_milb:
    for product in clients['product_current'].unique():
        all_products.append(product)
    #print(str(clients['lkupclientid'].unique())+ '\n' +str(clients['product_current'].unique()))

In [24]:
#all_products

In [22]:
for item in all_products:
    print(f'{item} - {all_products.count(item)}') 
#Full Season, Individual, Group, Half Season  

Full Season - 18
None - 20
Mini Plans / Flex Plans - 1
Individual - 18
Half Season - 13
Quarter Season - 2
Groups / Picnics - 1
Hospitality - 2
Mini Plan - 9
None - 20
Individual - 18
Group - 17
Picnic - 1
Half Season - 13
Full Season - 18
None - 20
Individual - 18
Quarter Season - 2
Full Season - 18
Group - 17
Half Season - 13
Mini Plan - 9
None - 20
Individual - 18
Mini Plan - 9
Group - 17
Full Season - 18
Flex Plan - 4
Half Season - 13
None - 20
Group - 17
Individual - 18
Mini Plan - 9
Full Season - 18
Half Season - 13
Mini Plan - 9
None - 20
Individual - 18
Group - 17
Half Season - 13
Full Season - 18
None - 20
None - 20
Full Season - 18
Individual - 18
Flex Plan - 4
Group - 17
Mini Plan - 9
Half Season - 13
None - 20
Individual - 18
Group - 17
Mini Plan - 9
Half Season - 13
Club Seats - 1
Full Season - 18
None - 20
Individual - 18
Partial Plan - 1
Full Season - 18
Group - 17
Half Season - 13
Individual - 18
None - 20
Group - 17
Full Season - 18
Paw-tial Plan - 1
Half Season - 13
F